In [20]:
from utils import series_ts_extraction_fns
import pandas as pd
import numpy as np

In [28]:
!pip install datasets

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 33.3 kB/s eta 0:00:0031m44.5 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 389.9 kB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 254.3 kB/s eta 0:00:0031m1.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 417.2 kB/s eta 0:00:0031m1.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 577.5 kB/s eta 0:00:00 eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 253.9 kB/s eta 0:00:00m eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 145.6 kB/s eta 0:00:001m381.2 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 526.2 kB/s e

In [2]:
from datasets import load_dataset
dataset = load_dataset("Monash-University/monash_tsf", "weather")

weather_dataset.zip:   0%|          | 0.00/38.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3010 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3010 [00:00<?, ? examples/s]

In [7]:
from joblib import Parallel, delayed

results = Parallel(n_jobs=len(series_ts_extraction_fns), backend="loky")(
    delayed(lambda f: (f.__name__, f(y)))(f) for f in series_ts_extraction_fns
)

/home/snape/miniconda3/envs/mlenv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
/home/snape/projects/Benchmarking-Hyper-Parameter-Tuning-Methods-for-Time-Series-Forecasting/utils.py:2322: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = data_close.pct_change().dropna()
/home/snape/projects/Benchmarking-Hyper-Parameter-Tuning-Methods-for-Time-Series-Forecasting/utils.py:2346: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future 

In [10]:
# Ensure X and y have the same number of rows by resetting indices
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

# Now create the mask after resetting the indices
mask = y.notna().values

# Ensure that the mask length matches the length of X
if len(mask) != len(X):
    raise ValueError("The length of the mask does not match the number of rows in X")

# Concatenate X and results, then apply the mask
X_fin_features = pd.concat([X, pd.DataFrame({name: series.squeeze() for name, series in results})], axis=1).iloc[mask].reset_index(drop=True)

# Filter y accordingly
y = y.iloc[mask].reset_index(drop=True)

# Continue with your numeric columns and imputation process
numeric_cols = X_fin_features.select_dtypes(include=[np.number]).columns
X_numeric = X_fin_features[numeric_cols].replace([np.inf, -np.inf], np.nan)
imputer = KNNImputer(n_neighbors=100, add_indicator=True, weights="distance")
X_imputed_numeric = pd.DataFrame(imputer.fit_transform(X_numeric), columns=numeric_cols)
X_fin_features[numeric_cols] = X_imputed_numeric


ValueError: The length of the mask does not match the number of rows in X